### Utilisation du fichier pickle

In [ ]:
import pickle
import pandas as pd
import re

with open('pickle/allocine_1.pickle', 'rb') as file:
    data = pickle.load(file)

### Connection bdd

In [ ]:
import psycopg2
from dotenv import load_dotenv
load_dotenv('.config')
import os

conn_mydb = psycopg2.connect(
    host = os.getenv('DB_HOST'),
    port = os.getenv('DB_PORT'),
    database = os.getenv('DB_NAME'),
    user = os.getenv('DB_USER'),
    password = os.getenv('DB_PASSWORD')
)

cur = conn_mydb.cursor()

### Ajout en bdd

In [ ]:
films_titles = [('Forrest Gump', '10568'), ('La Liste de Schindler', '9393'), ('La Ligne verte', '22779'), ('12 hommes en colère', '4063'), ('Le Parrain', '1628'), ('Les Evadés', '11736'), ('Le Seigneur des anneaux : le retour du roi', '39187'), ('Le Roi Lion', '10862'), ("Vol au-dessus d'un nid de coucou", '2072'), ('The Dark Knight, Le Chevalier Noir', '115362'), ("Il était une fois dans l'Ouest", '2021'), ('Pulp Fiction', '10126'), ('Le Bon, la brute et le truand', '19395'), ('Il était une fois en Amérique', '1941'), ('Interstellar', '114782'), ('Gladiator', '24944'), ('Django Unchained', '190918'), ("Le Seigneur des anneaux : la communauté de l'anneau", '27070'), ('Le Seigneur des anneaux : les deux tours', '39186'), ('Gran Torino', '135063')]
# data film
sql_add_film = 'INSERT INTO film (title, reference) VALUES (%s, %s) ON CONFLICT (reference) DO NOTHING;'

# data spectator_critics
# Faire une boucle pour ajouter chaque valeur 
sql_add_spec_critics = ' INSERT INTO spectator_critics (text, stars, publication_date, id_film) VALUES (%s, %s, %s, %s);'

sql_fk_film = 'SELECT id_film FROM film WHERE reference = %s'
        
for i in films_titles:
    title = i[0]
    id_film = int(i[1])
    cur.execute(sql_add_film, (title, id_film))
    conn_mydb.commit()

print('tout les film add')

selected_columns = data[['rating', 'review', 'date','film-url']]
pattern = r'film/fichefilm-(\d+)'
for index, row in selected_columns.iterrows():
    rating = row['rating']
    review = row['review']
    date = row['date']
    url = row['film-url']
    match = re.search(pattern, url)
    if match:
        fk_film = match.group(1)

    cur.execute(sql_fk_film,(fk_film,))
    fk_film = cur.fetchone()[0]

    cur.execute(sql_add_spec_critics, (review, rating, date, int(fk_film)))
    conn_mydb.commit()
    
print('tout les com add')


# cur.execute('INSERT INTO film (title, reference) VALUES(%s, %s)', ('test', 1))
conn_mydb.commit()
# cur.close()
# conn_mydb.close()